In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn

In [2]:
class WideAndDeepModel(tf.keras.Model):
  def __init__(self, units = 30 , activation = 'relu' ,**kwargs):
    super().__init__(**kwargs)
    self.norm_layer_wide = tf.keras.layers.Normalization()
    self.norm_layer_deep = tf.keras.layers.Normalization()
    self.hidden1 = tf.keras.layers.Dense(units, activation = activation)
    self.hidden2 = tf.keras.layers.Dense(units, activation = activation)
    self.main_output = tf.keras.layers.Dense(1)
    self.aux_output = tf.keras.layers.Dense(1)


  def call(self, inputs):
    input_wide, input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)
    norm_deep = self.norm_layer_deep(input_deep)
    hidden1 = self.hidden1(norm_deep)
    hidden2 = self.hidden2(hidden1)
    concat = tf.keras.layers.concatenate([norm_wide, hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output, aux_output

In [3]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('checkpointsl1', save_weights_only = True)

class ShowLossRatios(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    ratio = logs['val_loss']/logs['loss']
    print(f'ratio = {ratio}')

In [4]:
input_wide = tf.keras.layers.Input(shape = [5])
input_deep = tf.keras.layers.Input(shape = [6])
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation = tf.keras.activations.relu)(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation = tf.keras.activations.relu)(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
main_output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs = [input_wide, input_deep], outputs = [main_output, aux_output])